In [1]:
import Pkg.instantiate
instantiate()
using BenchmarkTools: @ballocated
using LinearAlgebra: I, norm, istriu, triu, qr, eigvals, diag
using CairoMakie

In [243]:
#Part A

# creates Identity matrix of correct size
m = 8
T = 1.0 * I(m)

# populates diagonal with correct eigenvalues
for i=1:size(T, 1)
    T[m-i+1,m-i+1] = 3^(i-1)
end

# generates random eigenvector matrix
L = randn(m, m)

# creates matrix A from eigendecomposition
A = L * T * inv(L)

# checks correct eigenvalues
display(eigvals(A))

8-element Vector{Float64}:
    0.9999999999999868
    3.0000000000002673
    9.000000000000089
   26.999999999999545
   80.99999999999987
  242.99999999999994
  728.9999999999999
 2187.0000000000036

In [245]:
#Part B

# Initializes Qk and Rk in QR factorization. Rk has n more dimensions to save the old iterations of Rk matrices
n = 5
Qk = randn(m, m)
Rk = zeros(n, m, m)

# run for n iterations (5)
for k=1:n
    # Update Qk
    Qk = A * Qk

    # Calculate QR factorization 
    Qk, Rk[k,:,:] = qr(Qk)

    # Grab eigenvalues after each iteration
    d = round.(diag(Rk[k,:,:]), sigdigits=4)
    println("itr $k: diag $d")
end

display(diag(T))

8-element Vector{Float64}:
 2187.0
  729.0
  243.0
   81.0
   27.0
    9.0
    3.0
    1.0

itr 1: diag [-5458.0, -1965.0, -1562.0, 418.9, -6.651, -19.84, 1.885, -2.172]
itr 2: diag [2079.0, 676.2, 255.9, 99.24, 18.0, 12.92, -3.621, -0.7608]
itr 3: diag [2164.0, -718.2, 243.3, 86.65, 28.5, -8.46, -3.22, -0.8993]
itr 4: diag [2181.0, 727.2, 242.8, 82.68, 28.23, 8.59, 3.071, 0.9648]
itr 5: diag [2185.0, 728.7, 242.9, 81.53, 27.46, 8.842, 3.023, 0.9881]


In [277]:
#Part A
m = 8
T = 1.0 * I(m)
for i=1:size(T, 1)
    T[m-i+1,m-i+1] = 3^(i-1)
end
L = randn(m, m)
A = L * T * inv(L)
# A = T

#Part B
Qk = I(m)
n = 5
Rk = zeros(n, m, m)
Ak = zeros(n, m, m)

for k=1:n
    Qk = A * Qk
    Qk, Rk[k,:,:] = qr(Qk)
    Ak[k, :, :] = Qk' * A * Qk

    d = round.(diag(Rk[k,:,:]), sigdigits=4)
    # println("itr $k: diag $d")
end
# display(diag(T))


# Part C
# Calculate eigenvalue estimate error
p_max = 3
eig_error = zeros(p_max, n)
for p=1:p_max
    for k=1:n
        eig_error[p, k] = abs((diag(Rk[k,:,:])[p] - T[p,p]) / T[p, p])
    end
end

# Calculate theoretical convergence
conv_theory = zeros(m, n)
for j = 1:m
    for i = 1:n
        if j==m
            conv_theory[j, i] = (T[j, j] / T[j-1, j-1]) ^ i
        else
            conv_theory[j, i] = (T[j+1, j+1] / T[j, j]) ^ i
        end
    end
end

# Plot Error
k_list = 1:n
error_plot = scatter(k_list, eig_error[1,:], label="λ_1 = $(T[1,1])", axis=(yscale=log10, xlabel="iteration, k", ylabel="Normalized Eigenvalue Error"))
for p=2:p_max
    scatter!(k_list, eig_error[p,:], label="λ_$p = $(T[p,p])")
end
scatter!(k_list, conv_theory[1,:], label="Theory")
axislegend(position=:rt)
save("eig_error.pdf", error_plot)

#Part D
# Calculate the norm of the submatrix
p = 4
mat_error = zeros(n,1)
for k = 1:n
    mat_error[k] = norm(Ak[k, (p+1):m, 1:p])
end

# Plot the error and convergence
mat_error_plot = scatter(1:n, mat_error[:], label="Experimental", axis=(yscale=log10, xlabel="iteration, k", ylabel="Norm"))
scatter!(k_list, conv_theory[1,:], label="Theory")
axislegend(position=:rt)
save("mat_error.pdf", mat_error_plot)

CairoMakie.Screen{PDF}


CairoMakie.Screen{PDF}
